## Web APIs

Utilizando uma API web para ler dados de um CEP

In [1]:
import requests
cep = '88050001'
url = 'https://viacep.com.br/ws/{cep}/json/'
url = url.format(cep=cep)
r = requests.get(url)
print(r.text)

{
  "cep": "88050-001",
  "logradouro": "Rodovia José Carlos Daux",
  "complemento": "de 9000/9001 a 11999/12000",
  "bairro": "Santo Antônio de Lisboa",
  "localidade": "Florianópolis",
  "uf": "SC",
  "unidade": "",
  "ibge": "4205407",
  "gia": ""
}


## HTTP

Podemos brincar um pouco com o protocolo HTTP usando o site [httpbin](http://httpbin.org/) 

### Fazendo um GET e inspecionando os cabeçalhos de resposta básicos

In [2]:
# Fazendo um GET no site
resposta = requests.get('http://httpbin.org/')

# Verificando o código HTTP da resposta
resposta.status_code

200

In [3]:
# Imprimindo os cabeçalhos da resposta
print(resposta.headers)

{'Connection': 'keep-alive', 'Date': 'Mon, 10 Oct 2016 20:33:08 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Server': 'nginx', 'Content-Length': '12150', 'Content-Type': 'text/html; charset=utf-8'}


In [ ]:
# Qual o tipo do conteúdo da resposta?
print(resposta.headers['Content-Type'])

In [ ]:
# Qual o tamanho da resposta?
print(resposta.headers['Content-Length'])

In [ ]:
# Fazendo um GET em https://api.ipify.org/
meu_ip = requests.get('https://api.ipify.org/')

# Imprimindo o conteúdo da resposta
print(meu_ip.text)

### Negociando o tipo do conteúdo usando cabeçalhos de pedido

In [ ]:
# Pedindo por uma imagem, sem especificar nada
imagem = requests.get('http://httpbin.org/image')

# Qual o tipo da imagem retornado?
print(imagem.headers['Content-Type'])

In [ ]:
# application/json não é um tipo de imagem! O que aconteceu?
print(imagem.text)

In [ ]:
# Qual foi o código de resposta?
print(imagem.status_code)

In [ ]:
# Precisamos inserir um cabeçalho, especificando que queremos uma imagem
cabeçalho = {'accept': 'image/webp'}  # Cabeçalho que pede pelo tipo PNG
imagem = requests.get('http://httpbin.org/image', headers=cabeçalho)

# Qual o tipo agora?
print(imagem.headers['Content-Type'])

In [ ]:
# Qual o código da resposta?
print(imagem.status_code)

In [ ]:
# Qual o tamanho da resposta?
print(imagem.headers['Content-Length'])

In [ ]:
# Exibindo a imagem!
from IPython.display import Image
Image(imagem.content)

### Outros códigos de resposta

In [ ]:
# Tentando acessar uma página proibida!
página_proibida = requests.get('http://httpbin.org/status/403')

print(página_proibida.status_code)

In [ ]:
# E uma página que não existe?
página_inexistente = requests.get('http://httpbin.org/blablabla')
print(página_inexistente.status_code)

In [ ]:
# Tentando acessar uma página que me redireciona para outra.
# Utilizei a flag `allow_redirects=False` para que a biblioteca `requests` não me redirecione automaticamente
# Experimente fazer o pedido alterando a flag para True!
redirecionamento = requests.get('http://httpbin.org/redirect-to?url=http://2016.pythonbrasil.org.br/',
                                allow_redirects=False)

# Qual foi o código de resposta?
print(redirecionamento.status_code)

In [ ]:
# Quais os cabeçalhos da resposta?
print(redirecionamento.headers)

In [ ]:
# Observe o cabeçalho `Location`!
print(redirecionamento.headers['Location'])

### Enviando parâmetros e dados via HTTP

In [ ]:
# Nós podemos especificar parâmetros ao fazer um pedido do tipo GET
parâmetros = {
    'data': '22/10/2012',
    'local': 'Brasil'
}

r = requests.get('http://httpbin.org/get', params=parâmetros)

# O HTTP Bin nos retorna no corpo da resposta os parâmetros que a gente enviou
print(r.text)

In [ ]:
# Utilizando o verbo POST, podemos enviar dados para serem processados pela aplicação
dados = {
    'nome': 'João',
    'idade': 20
}
r = requests.post('http://httpbin.org/post', data=dados)

# Assim como no caso do GET, o HTTP Bin retorna os dados que enviamos no POST
print(r.text)

## JSON

### Serialização e Desserialização

In [ ]:
from collections import namedtuple

# Criando uma classe simples para armazenar dados de uma pessoa
Pessoa = namedtuple('Pessoa', ['nome', 'idade', 'altura', 'bandas_favoritas', 'contatos'])

In [ ]:
# Criando uma nova instância para armazenar os dados de José
josé = Pessoa(nome='José', idade=30, altura=1.76,
              bandas_favoritas=['U2', 'Creedence'],
              contatos={'email': 'jose@jose.com'})

In [ ]:
# `josé` é do tipo Pessoa!
type(josé)

In [ ]:
# Inspecionando o que temos em `josé`
print(josé)

In [ ]:
# Convertendo `josé` para JSON
import json
j = json.dumps(josé._asdict())
print(j)

In [ ]:
# `j` agora é uma string!
type(j)

Por ser uma string, podemos transmiti-la, armazená-la, etc

Esse processo de converter uma classe em uma entidade transmitível como uma string é chamado de **serialização**

In [ ]:
# Restaurando um objeto com base em `j`
dados_novo_josé = json.loads(j)

# `novo_josé` é um dicionário contendo os dados de `josé`
print(dados_novo_josé)

In [ ]:
type(dados_novo_josé)

In [ ]:
novo_josé = Pessoa(**dados_novo_josé)

In [ ]:
type(novo_josé)

In [ ]:
print(novo_josé)

In [ ]:
# Os dois josés são iguais?
novo_josé == josé

Esse processo de converter o valor armazenado de volta em um objeto é chamado de **desserialização**

### Manipulando JSON que vem da web

In [ ]:
# Fazendo o mesmo request de CEP do começo
import requests
cep = '88050001'
url = 'https://viacep.com.br/ws/{cep}/json/'
url = url.format(cep=cep)
r = requests.get(url)
print(r.text)

In [ ]:
# Lendo os dados como JSON para um dicionário!
dados = json.loads(r.text)

In [ ]:
# Qual o tipo de dados que temos agora?
type(dados)

In [ ]:
# Imprimindo nossos dados
print(dados)

In [ ]:
# Como agora temos um dicionário, podemos acessar suas chaves
print(dados['localidade'])

In [ ]:
print(dados['bairro'])

In [1]:
print('Você está na cidade de {cidade}-{estado}!'.format(cidade=dados['localidade'], estado=dados['uf']))

NameError: name 'dados' is not defined

### OAUTH2

In [ ]:
import requests
cabeçalho = {
    'Authorization': "OAuth oauth_consumer_key='{key}', oauth_nonce='{nonce}', oauth_signature='{sig}', oauth_signature_method='HMAC-SHA1', oauth_timestamp='{timestamp}',oauth_version='1.0''"
}

key = 'qinsVbPWmJkRbwB5rJ6Aus8vk'
nonce = '517d921228dd9277b0468af3c4838c9c'
sig = 'H3OsYv9h8bQ0XUKcE%2FJbFl8AC%2B8%3D'
timestamp = '1476326207'

cabeçalho['Autorization'] = cabeçalho['Authorization'].format(key=key, nonce=nonce, sig=sig, timestamp=timestamp)
tweets = requests.get('https://api.twitter.com/1.1/search/tweets.json?q=pythonbrasil',
                     headers=cabeçalho)